In [1]:
import sys
sys.path.append('/home/btheilma/code/gentnerlab/ephys-analysis/')
from ephys import rasters, core, events

import matplotlib.pyplot as plt
import numpy as np
%pylab inline


ImportError: No module named ephys

In [ ]:
block_path = '/mnt/cube/ephys-example-data/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'

spikes = core.load_spikes(block_path)
trials = events.load_trials(block_path)
fs = core.load_fs(block_path)

stims = trials['stimulus'].unique()
f, pltaxes = plt.subplots(4, 4, sharey=True, figsize=(22,18))

clusterID = 8
raster_window = [-2, 2]
rec = 0

for ind, stim in enumerate(stims):
    ax = pltaxes.flatten()[ind]
    rasters.plot_raster_cell_stim(spikes, trials, clusterID, stim, 
                                  raster_window, rec, fs, ax=ax)
    ax.set_title('Cell: {} Stim: {}'.format(str(8), stim))
    

In [ ]:
pltaxes.flatten()[0]

In [ ]:
ax.set_title('test')